# SC1015 : DSAI Final Project FCE2 Team 1

## Fake News Analyses & Detection using NLP, Machine Learning & Neural Networks

##### Akshay Narayanan, Maanya, Aryan

In [6]:
# This block has been added since this notebook was written on Kaggle.
#This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-real-news/news_dataset.csv
/kaggle/input/abcdef/all_validate.tsv
/kaggle/input/abcdef/all_train (1).tsv
/kaggle/input/abcdef/all_test_public.tsv
/kaggle/input/liar101/test.tsv
/kaggle/input/liar101/train.tsv
/kaggle/input/liar101/valid.tsv
/kaggle/input/fakenews101/all_train.tsv


#### Importing the necessary libraries to **handle data** in python.

In [7]:
import pandas as pd #Handling datasets
import numpy as np #Handling large scale numeric operations
import matplotlib.pyplot as plt #For visualizations

#### Reading each of the fake news datasets for fake news detection

In [10]:
df1 = pd.read_csv("/kaggle/input/liar101/train.tsv",sep="\t") #10,239
df2 = pd.read_csv("/kaggle/input/liar101/test.tsv",sep="\t") #1266
df3 = pd.read_csv("/kaggle/input/abcdef/all_train (1).tsv",sep="\t") #286054
df4 = pd.read_csv("/kaggle/input/abcdef/all_test_public.tsv",sep="\t") #30022
df6 = pd.read_csv("/kaggle/input/fake-real-news/news_dataset.csv") #3271
df7 = pd.read_csv("https://docs.google.com/spreadsheets/d/1Dm8E2McVkR1teBicAN5N9kLSIMXADQnBBqJ9LDhnctc/export?format=csv&gid=647897913") #54170

#### Observing each dataset one by one

#### Steps Include
##### 1. Noting down the structure of the dataset.
##### 2. Number of datapoints, quantiles and other basic statistical measures.
##### 3. Finding out if there are missing values and handling the same.
##### 4. Need for normalization? [Mostly no since this is a text based use case of Data Science]
##### 5. Formatting dataset into a standard structure for combining it later.

In [11]:
df1.head()

,2635.json,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0.1,0.2,0.3,a mailer
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [12]:
df1 = df1.rename(columns={'false': 'Label', 'Says the Annies List political group supports third-trimester abortions on demand.': 'Text', 'abortion': 'Subject','Texas':'Country'})
df1 = df1.drop(["2635.json","dwayne-bohac","State representative","republican","0","1","0.1","0.2","0.3","a mailer"],axis=1)
labdict = {"pants-fire":0,"false":1,"barely-true":2,"half-true":3,"mostly-true":4,"true":5}
dfa['Label'] = dfa['Label'].map(labdict)

In [18]:
df1.head()

,Label,Text,Subject,Country
0,3,When did the decline of coal start? It started...,"energy,history,job-accomplishments",Virginia
1,4,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,Illinois
2,1,Health care reform legislation is likely to ma...,health-care,NaN
3,3,The economic turnaround started at the end of ...,"economy,jobs",Florida
4,5,The Chicago Bears have had more starting quart...,education,Wisconsin


In [19]:
df1.describe()

,Label
count,10239.000000
mean,2.722141
std,1.564417
min,0.000000
25%,1.000000
50%,3.000000
75%,4.000000
max,5.000000


In [11]:
dfa.head()

,Text,Subject,label
0,Wisconsin is on pace to double the number of l...,jobs,1
1,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",1
2,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",3
3,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",0
4,Over the past five years the federal governmen...,"federal-budget,pensions,retirement",5


In [20]:
df2.head()

,11972.json,true,Building a wall on the U.S.-Mexico border will take literally years.,immigration,rick-perry,Governor,Texas,republican,30,30.1,42,23,18,Radio interview
0,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference
1,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.
2,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show
3,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video
4,5962.json,true,Over the past five years the federal governmen...,"federal-budget,pensions,retirement",brendan-doherty,NaN,Rhode Island,republican,1,2,1,1,0,a campaign website


In [21]:
df2 = df2.rename(columns={'true': 'Label', 'Building a wall on the U.S.-Mexico border will take literally years.': 'Text', 'immigration': 'Subject','Texas':'Country'})
df2 = df2.drop(["11972.json","rick-perry","Governor","republican","30","30.1","42","23","18","Radio interview"],axis=1)
labdict = {"pants-fire":0,"false":1,"barely-true":2,"half-true":3,"mostly-true":4,"true":5}
df2['Label'] = df2['Label'].map(labdict)

In [22]:
df2.head()

,Label,Text,Subject,Country
0,1,Wisconsin is on pace to double the number of l...,jobs,Wisconsin
1,1,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",New York
2,3,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",Oregon
3,0,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",Wisconsin
4,5,Over the past five years the federal governmen...,"federal-budget,pensions,retirement",Rhode Island


In [23]:
df2.describe()

,Label
count,1266.000000
mean,2.738547
std,1.544711
min,0.000000
25%,1.000000
50%,3.000000
75%,4.000000
max,5.000000


In [24]:
df3.head()

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,author,clean_title,created_utc,domain,hasImage,id,image_url,linked_submission_id,num_comments,score,subreddit,title,upvote_ratio,2_way_label,3_way_label,6_way_label
0,0,0,NaN,NaN,Alexithymia,my walgreens offbrand mucinex was engraved wit...,1.551641e+09,i.imgur.com,True,awxhir,https://external-preview.redd.it/WylDbZrnbvZdB...,NaN,2.0,12,mildlyinteresting,My Walgreens offbrand Mucinex was engraved wit...,0.84,1,0,0
1,1,1,155885.0,714550.0,RickSisco,NaN,1.443822e+09,NaN,True,cvm5uy4,http://i.imgur.com/yxrkYT8.jpg,3n7fld,NaN,5,psbattle_artwork,NaN,NaN,0,2,4
2,2,2,NaN,NaN,VIDCAs17,this concerned sink with a tiny hat,1.534727e+09,i.redd.it,True,98pbid,https://preview.redd.it/wsfx0gp0f5h11.jpg?widt...,NaN,2.0,119,pareidolia,This concerned sink with a tiny hat,0.99,0,2,2
3,3,3,NaN,NaN,prometheus1123,hackers leak emails from uae ambassador to us,1.496511e+09,aljazeera.com,True,6f2cy5,https://external-preview.redd.it/6fNhdbc6K1vFA...,NaN,1.0,44,neutralnews,Hackers leak emails from UAE ambassador to US,0.92,1,0,0
4,4,4,282323.0,1228398.0,NaN,NaN,1.378792e+09,NaN,True,cc5cbon,http://i.imgur.com/M8KTWMx.jpg,1lz1q0,NaN,3,psbattle_artwork,NaN,NaN,0,2,4


In [25]:
df3.describe()

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,created_utc,num_comments,score,upvote_ratio,2_way_label,3_way_label,6_way_label
count,878218.000000,878218.000000,242100.000000,2.421000e+05,8.782180e+05,636118.00000,878218.000000,636118.000000,878218.000000,878218.000000,878218.000000
mean,439108.500000,439108.500000,171440.742623,7.611963e+05,1.483381e+09,17.88704,312.269706,0.853566,0.455961,1.069485,1.709166
std,253519.843691,253519.843691,95582.804941,4.116242e+05,6.707227e+07,124.07375,2621.030757,0.116801,0.498057,0.988220,1.785979
min,0.000000,0.000000,2.000000,9.000000e+00,1.212297e+09,0.00000,-950.000000,0.500000,0.000000,0.000000,0.000000
25%,219554.250000,219554.250000,90829.750000,4.090352e+05,1.428682e+09,1.00000,3.000000,0.770000,0.000000,0.000000,0.000000
50%,439108.500000,439108.500000,172780.500000,7.873315e+05,1.493982e+09,2.00000,10.000000,0.870000,0.000000,2.000000,1.000000
75%,658662.750000,658662.750000,253815.250000,1.112994e+06,1.550847e+09,7.00000,34.000000,0.950000,1.000000,2.000000,4.000000
max,878217.000000,878217.000000,335339.000000,1.444053e+06,1.573859e+09,10783.00000,137179.000000,1.000000,1.000000,2.000000,5.000000


In [26]:
df3 = df3.drop(["Unnamed: 0.2","Unnamed: 0","Unnamed: 0.1","Unnamed: 0.1.1","author","created_utc","domain","hasImage","id",
                "image_url","linked_submission_id","num_comments",
               "score","subreddit","upvote_ratio","3_way_label"],axis=1)
df3 = df3.rename(columns={'6_way_label': 'Label','2_way_label':'tf', 'title': 'Text'})

In [27]:
df3.head()

,clean_title,Text,tf,Label
0,my walgreens offbrand mucinex was engraved wit...,My Walgreens offbrand Mucinex was engraved wit...,1,0
1,NaN,NaN,0,4
2,this concerned sink with a tiny hat,This concerned sink with a tiny hat,0,2
3,hackers leak emails from uae ambassador to us,Hackers leak emails from UAE ambassador to US,1,0
4,NaN,NaN,0,4


In [34]:
def count_words(text):
    return len(text.split())

df3 = df3.dropna()
df3 = df3[df3['Text'].apply(count_words) >= 10]

In [35]:
df3.describe()

,tf,Label
count,286054.000000,286054.000000
mean,0.650356,0.991128
std,0.476858,1.559447
min,0.000000,0.000000
25%,0.000000,0.000000
50%,1.000000,0.000000
75%,1.000000,2.000000
max,1.000000,5.000000


In [36]:
df4.head()

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,author,clean_title,created_utc,domain,hasImage,id,image_url,linked_submission_id,num_comments,score,subreddit,title,upvote_ratio,2_way_label,3_way_label,6_way_label
0,0,0,NaN,NaN,buzzly6,virginia first lady criticized for handing cot...,1.551316e+09,philly.com,False,avkxum,NaN,NaN,2.0,16,nottheonion,Virginia first lady criticized for handing cot...,0.69,1,0,0
1,1,1,NaN,NaN,NaN,jason falconer reluctant hero stopped mall att...,1.474326e+09,cnn.it,False,53kdz3,NaN,NaN,0.0,7,upliftingnews,Jason Falconer: Reluctant hero stopped mall at...,0.77,1,0,0
2,2,2,75691.0,335711.0,myternity,NaN,1.497067e+09,NaN,True,diperbi,http://i.imgur.com/cSIuEVF.jpg,6gc83w,NaN,3,psbattle_artwork,NaN,NaN,0,2,4
3,3,3,NaN,NaN,NaN,woman bites camels testicles to save herself a...,1.569178e+09,wbrz.com,False,d7upss,NaN,NaN,3.0,7,nottheonion,Woman bites camel’s testicles to save herself ...,1.00,1,0,0
4,4,4,190207.0,862707.0,trustbytrust,stargazer,1.425139e+09,NaN,True,cozywbv,http://i.imgur.com/BruWKDi.jpg,2xct9d,NaN,3,psbattle_artwork,stargazer,NaN,0,2,4


In [39]:
df4 = df4.drop(["Unnamed: 0.2","Unnamed: 0","Unnamed: 0.1","Unnamed: 0.1.1","author","created_utc","domain","hasImage","id",
                "image_url","linked_submission_id","num_comments",
               "score","subreddit","upvote_ratio","3_way_label"],axis=1)
df4 = df4.rename(columns={'6_way_label': 'Label','2_way_label':'tf', 'title': 'Text'})

In [40]:
df4 = df4.dropna()
df4 = df4[df4['Text'].apply(count_words) >= 10]

In [41]:
df4.describe()

,tf,Label
count,30022.000000,30022.00000
mean,0.650889,0.98641
std,0.476697,1.55546
min,0.000000,0.00000
25%,0.000000,0.00000
50%,1.000000,0.00000
75%,1.000000,2.00000
max,1.000000,5.00000


In [42]:
df6.head()

,label,text
0,REAL,Payal has accused filmmaker Anurag Kashyap of ...
1,FAKE,A four-minute-long video of a woman criticisin...
2,FAKE,"Republic Poll, a fake Twitter account imitatin..."
3,REAL,"Delhi teen finds place on UN green list, turns..."
4,REAL,Delhi: A high-level meeting underway at reside...


In [49]:
df6['Label'] = df6['label'].replace({'REAL': 1, 'FAKE': 0})
df6 = df6.rename(columns={'text': 'Text'})
df6 = df6.dropna()

/tmp/ipykernel_33/3251431125.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df6['Label'] = df6['label'].replace({'REAL': 1, 'FAKE': 0})


In [50]:
df6.describe()

,Label
count,3721.000000
mean,0.497178
std,0.500059
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [51]:
df7.head()

,id,Statement,Image,Web,Category,Date,Label
0,2,"WHO praises India's Aarogya Setu app, says it ...",https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,COVID-19,Oct-20,TRUE
1,3,"In Delhi, Deputy US Secretary of State Stephen...",https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,VIOLENCE,Oct-20,TRUE
2,4,LAC tensions: China's strategy behind delibera...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,TERROR,Oct-20,TRUE
3,5,India has signed 250 documents on Space cooper...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,COVID-19,Oct-20,TRUE
4,6,Tamil Nadu chief minister's mother passes away...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,ELECTION,Oct-20,TRUE


In [56]:
df7 = df7.drop(["id","Image","Web","Date"],axis=1)
df7 = df7.rename(columns={'Statement': 'Text','Category':'Subject'})

In [71]:
df7 = df7.dropna()
df7.drop_duplicates(inplace=True)
df7['Label'] = df7['Label'].replace({'TRUE': 1, 'FALSE': 0})

In [72]:
df7.describe()

,Text,Subject,Label
count,54170,54170,54170
unique,50974,9,2
top,"DMK lacks leadership, will split after 2021 TN...",GOVERNMENT,1
freq,3,10366,35958


In [79]:
df1["Country"] = "USA"
df1.head()

,Label,Text,Subject,Country
0,3,When did the decline of coal start? It started...,"energy,history,job-accomplishments",USA
1,4,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,USA
2,1,Health care reform legislation is likely to ma...,health-care,USA
3,3,The economic turnaround started at the end of ...,"economy,jobs",USA
4,5,The Chicago Bears have had more starting quart...,education,USA


In [80]:
df2["Country"] = "USA"
df2.head()

,Label,Text,Subject,Country
0,1,Wisconsin is on pace to double the number of l...,jobs,USA
1,1,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",USA
2,3,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",USA
3,0,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",USA
4,5,Over the past five years the federal governmen...,"federal-budget,pensions,retirement",USA


In [91]:
df3["Country"] = "USA"
df3["Text"] = df3["clean_title"]
df3 = df3.drop(["clean_title","tf"],axis=1)
df3.head()

,Text,Label,Country
0,my walgreens offbrand mucinex was engraved wit...,0,USA
9,bride and groom exchange vows after fatal shoo...,0,USA
12,rabbi meat from cloned pig could be kosher for...,0,USA
14,jesus christ converting local teens to christi...,2,USA
16,victory the great european crusade vichy france,5,USA


In [92]:
df4["Country"] = "USA"
df4["Text"] = df4["clean_title"]
df4 = df4.drop(["clean_title","tf"],axis=1)
df4.head()

,Text,Label,Country
0,virginia first lady criticized for handing cot...,0,USA
3,woman bites camels testicles to save herself a...,0,USA
5,driving scares me a few little kittens down a ...,3,USA
7,penn jillette says trump apprentice tapes exis...,0,USA
11,as trump accuses iran he has one problem his o...,0,USA


In [83]:
df6["Country"] = "India"
df6.head()

,label,Text,Label,Country
0,REAL,Payal has accused filmmaker Anurag Kashyap of ...,1,India
1,FAKE,A four-minute-long video of a woman criticisin...,0,India
2,FAKE,"Republic Poll, a fake Twitter account imitatin...",0,India
3,REAL,"Delhi teen finds place on UN green list, turns...",1,India
4,REAL,Delhi: A high-level meeting underway at reside...,1,India


In [84]:
df7["Country"] = "India"
df7.head()

,Text,Subject,Label,Country
0,"WHO praises India's Aarogya Setu app, says it ...",COVID-19,1,India
1,"In Delhi, Deputy US Secretary of State Stephen...",VIOLENCE,1,India
2,LAC tensions: China's strategy behind delibera...,TERROR,1,India
3,India has signed 250 documents on Space cooper...,COVID-19,1,India
4,Tamil Nadu chief minister's mother passes away...,ELECTION,1,India


In [86]:
df_net_with_subject = pd.concat([df2,df1], ignore_index=True)

In [93]:
df_net_no_subject = pd.concat([df3,df4],ignore_index=True)

In [94]:
df_net_with_subject.to_csv('df_net_with_subject.csv', index=False)
df_net_no_subject.to_csv('df_net_no_subject.csv', index=False)
df7.to_csv('categories.csv', index=False)

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

X_train, X_test, y_train, y_test = train_test_split(df_net['Text'], df_net['label'], test_size=0.2, random_state=42)
max_words = 10000 

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


max_len = 100 
X_train_padded = pad_sequences(X_train_seq, maxlen=max_len)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len)


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


embedding_dim = 100
model = Sequential()
model.add(Embedding(max_words, embedding_dim))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 32 
epochs = 7  #Thala For a reason
model.fit(X_train_padded, y_train_encoded, batch_size=batch_size, epochs=epochs, validation_data=(X_test_padded, y_test_encoded))

loss, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


2024-04-18 03:15:26.313648: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 03:15:26.313797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 03:15:26.521679: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Epoch 1/7
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.2046 - loss: 1.7615

KeyboardInterrupt: 

In [ ]:
model.save('fakenews01.h5')

In [ ]:
def checknews(news):
    nnews = tokenizer.texts_to_sequences([news])
    nnnews = pad_sequences(nnews, maxlen=max_len)
    preditc =  model.predict(nnnews)
    return np.argmax(preditc)

In [ ]:
checknews("Godzilla destroyes world by the order of Vishnu")

In [ ]:
checknews("COVID 19 Pandemic kills many")

In [ ]:
checknews("Superman is going to grade our presentations intead of our tutor")

In [ ]:
checknews("NTU IS WASTE")

In [ ]:
checknews("Over the past five years the federal government has paid out $601 million in retirement and disability benefits to deceased former federal employees.")

In [ ]:
checknews("Donald Trump is against marriage equality. He wants to go back.")

In [ ]:
checknews("COVID-19: India's single-day spike drops to 55,342 as tally approaches 72 lakh")

In [ ]:
df2['Text'].head(10).tolist()

In [ ]:
checknews("Bedok area is currently experiencing an outbreak of shellfish poisoning due to neurotoxins. Some patients who came to the hospital were in critical condition. Therefore, please do not consume any shellfish like mussels, clams, scallops, oysters or geoduck for now. The Ministry of Health is still investigating the severity of the situation.")